In [6]:
#set up autoreload
%load_ext autoreload
%autoreload 2

import polars as pl
from pathlib import Path

import importlib  
fa_etl = importlib.import_module("fa-etl")

pl.Config(set_fmt_float="full")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
path_to_dir = "/Users/claireboyd/internships/mansueto/firstamerican-etl/"
merged = pl.scan_parquet(
    path_to_dir+f"dev/merged.parquet"
    ).with_columns([
    pl.when(
        pl.col('AssdTotalValue').is_null() | pl.col('MarketTotalValue').is_null() | pl.col('MarketTotalValue') == 0
        ).then(None
    ).otherwise(
        pl.col('AssdTotalValue') / pl.col('MarketTotalValue')
    ).alias('sales_ratio')
])

In [36]:
merged.schema

OrderedDict([('PropertyID', Int64),
             ('Year', Int64),
             ('AssdTotalValue', Int64),
             ('MarketTotalValue', Int64),
             ('ApprTotalValue', Int64),
             ('SitusLatitude', Float64),
             ('SitusLongitude', Float64),
             ('SitusFullStreetAddress', String),
             ('SitusCity', String),
             ('SitusState', String),
             ('SitusZIP5', String),
             ('FIPS', String),
             ('SitusCensusTract', String),
             ('SitusCensusBlock', String),
             ('SaleAmt', Int64),
             ('TaxAmt', Int64),
             ('TaxAmtAdjusted', Float64),
             ('sales_ratio', Float64)])

In [37]:
merged.head().collect(streaming=True)

PropertyID,Year,AssdTotalValue,MarketTotalValue,ApprTotalValue,SitusLatitude,SitusLongitude,SitusFullStreetAddress,SitusCity,SitusState,SitusZIP5,FIPS,SitusCensusTract,SitusCensusBlock,SaleAmt,TaxAmt,TaxAmtAdjusted,sales_ratio
i64,i64,i64,i64,i64,f64,f64,str,str,str,str,str,str,str,i64,i64,f64,f64
276,2018,17180,85900,null,32.662936,-86.438723,"""2677 HIGHWAY 143""","""DEATSVILLE""","""AL""","""36022""","""01001""","""020901""","""1036""",82000,46386,463.86,0.2
292,2018,20700,206900,null,32.577186,-86.548768,"""695 YOSEMITE PKWY""","""PRATTVILLE""","""AL""","""36067""","""01001""","""020902""","""2040""",215000,55890,558.9,0.1000483325277912
165458406,2023,28220,282200,null,32.445844,-86.415837,"""209 HEDGEFIELD DR""","""PRATTVILLE""","""AL""","""36066""","""01001""","""020501""","""1017""",302500,87482,874.82,0.1
165458255,2023,42180,210900,null,32.449623,-86.450068,"""582 SUNSET DR""","""PRATTVILLE""","""AL""","""36067""","""01001""","""020700""","""1007""",230000,130758,1307.58,0.2
165458247,2021,19920,199200,null,32.450092,-86.449283,"""589 SUNSET DR""","""PRATTVILLE""","""AL""","""36067""","""01001""","""020700""","""1019""",214900,57652,576.52,0.1


In [59]:
sales_ratio_summary = merged.filter(
    pl.col("sales_ratio").is_not_null()
).group_by(
    "FIPS", "Year"
).agg(
    pl.len().alias('sales_ratio_non_nulls_count'),
    # pl.mean('sales_ratio').alias('sales_ratio_mean'),
    # pl.median('sales_ratio').alias('sales_ratio_median')
).collect(streaming=True)

In [60]:
sales_ratio_summary

FIPS,Year,sales_ratio_non_nulls_count
str,i64,u32
"""01083""",2020,3165
"""05065""",2017,206
"""12073""",2016,6139
"""02275""",2020,13
"""05119""",2019,8068
…,…,…
"""01111""",2018,312
"""05049""",2017,205
"""05111""",2021,340


In [61]:
national_summary = merged.group_by(
    "FIPS", "Year"
).agg(
    pl.count('PropertyID').alias('TotalSales'),
    pl.mean('AssdTotalValue').alias('AssdTotalValue_mean'),
    pl.median('AssdTotalValue').alias('AssdTotalValue_median'),
    pl.mean('sales_ratio').alias('sales_ratio_mean'),
    pl.median('sales_ratio').alias('sales_ratio_median')
).collect(streaming=True)

In [68]:
national_summary.join(
    other=sales_ratio_summary,
    on=["FIPS", "Year"],
    how="left"
).with_columns([
    (pl.col("sales_ratio_non_nulls_count")/pl.col("TotalSales")).alias("freq_valid_sales_ratio")
]).write_csv("firstamerican_nationalsummary.csv")